In [2]:
#Import all the necessary modules
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn import metrics
#from sklearn.decomposition import PCA
from scipy.stats import zscore
import matplotlib.pyplot as plt 
# importing ploting libraries
import matplotlib.pyplot as plt   
#importing seaborn for statistical plots
import seaborn as sns
# To enable plotting graphs in Jupyter notebook
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [20]:
colnames = ['name','Fo(Hz)', 'Fhi(Hz)','Flo(Hz)','Jitter(%)', 'Jitter(Abs)', 'RAP','PPQ','DDP','Shimmer','Shimmer(dB)','APQ3','APQ5','APQ','DDA','NHR','HNR','status','RPDE','DFA','spread1','spread2','D2','PPE']
parkinsons_df = pd.read_csv("parkinsons.data",names=colnames)
parkinsons_df = parkinsons_df.drop(parkinsons_df.index[0])
parkinsons_df.head(5)

,name,Fo(Hz),Fhi(Hz),Flo(Hz),Jitter(%),Jitter(Abs),RAP,PPQ,DDP,Shimmer,...,DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
1,phon_R01_S01_1,119.99200,157.30200,74.99700,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.03300,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
2,phon_R01_S01_2,122.40000,148.65000,113.81900,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.08500,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
3,phon_R01_S01_3,116.68200,131.11100,111.55500,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.65100,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
4,phon_R01_S01_4,116.67600,137.87100,111.36600,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.64400,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
5,phon_R01_S01_5,116.01400,141.78100,110.65500,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.64900,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [38]:
# parkinsons_df = pd.get_dummies(parkinsons_df, drop_first = True)
parkinsons_df = parkinsons_df.iloc[:,2:24].astype(float)
#parkinsons_df = parkinsons_df.drop(['name'], axis = 1).astype(float)
# parkinsons_df = parkinsons_df.astype(float)

In [39]:
parkinsons_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 1 to 195
Data columns (total 20 columns):
Jitter(%)      195 non-null float64
Jitter(Abs)    195 non-null float64
RAP            195 non-null float64
PPQ            195 non-null float64
DDP            195 non-null float64
Shimmer        195 non-null float64
Shimmer(dB)    195 non-null float64
APQ3           195 non-null float64
APQ5           195 non-null float64
APQ            195 non-null float64
DDA            195 non-null float64
NHR            195 non-null float64
HNR            195 non-null float64
status         195 non-null float64
RPDE           195 non-null float64
DFA            195 non-null float64
spread1        195 non-null float64
spread2        195 non-null float64
D2             195 non-null float64
PPE            195 non-null float64
dtypes: float64(20)
memory usage: 32.0 KB


In [35]:
parkinsons_df.describe().T


,count,mean,std,min,25%,50%,75%,max
Fhi(Hz),195.0,197.104918,91.491548,102.145000,134.862500,175.829000,224.205500,592.030000
Flo(Hz),195.0,116.324631,43.521413,65.476000,84.291000,104.315000,140.018500,239.170000
Jitter(%),195.0,0.006220,0.004848,0.001680,0.003460,0.004940,0.007365,0.033160
Jitter(Abs),195.0,0.000044,0.000035,0.000007,0.000020,0.000030,0.000060,0.000260
RAP,195.0,0.003306,0.002968,0.000680,0.001660,0.002500,0.003835,0.021440
PPQ,195.0,0.003446,0.002759,0.000920,0.001860,0.002690,0.003955,0.019580
DDP,195.0,0.009920,0.008903,0.002040,0.004985,0.007490,0.011505,0.064330
Shimmer,195.0,0.029709,0.018857,0.009540,0.016505,0.022970,0.037885,0.119080
Shimmer(dB),195.0,0.282251,0.194877,0.085000,0.148500,0.221000,0.350000,1.302000
APQ3,195.0,0.015664,0.010153,0.004550,0.008245,0.012790,0.020265,0.056470


In [40]:
parkinsons_df.shape

(195, 20)

In [41]:
parkinsons_df.corr()

,Jitter(%),Jitter(Abs),RAP,PPQ,DDP,Shimmer,Shimmer(dB),APQ3,APQ5,APQ,DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
Jitter(%),1.000000,0.935714,0.990276,0.974256,0.990276,0.769063,0.804289,0.746625,0.725561,0.758255,0.746635,0.906959,-0.728165,0.278220,0.360673,0.098572,0.693577,0.385123,0.433434,0.721543
Jitter(Abs),0.935714,1.000000,0.922911,0.897778,0.922913,0.703322,0.716601,0.697153,0.648961,0.648793,0.697170,0.834972,-0.656810,0.338653,0.441839,0.175036,0.735779,0.388543,0.310694,0.748162
RAP,0.990276,0.922911,1.000000,0.957317,1.000000,0.759581,0.790652,0.744912,0.709927,0.737455,0.744919,0.919521,-0.721543,0.266668,0.342140,0.064083,0.648328,0.324407,0.426605,0.670999
PPQ,0.974256,0.897778,0.957317,1.000000,0.957319,0.797826,0.839239,0.763580,0.786780,0.804139,0.763592,0.844604,-0.731510,0.288698,0.333274,0.196301,0.716489,0.407605,0.412524,0.769647
DDP,0.990276,0.922913,1.000000,0.957319,1.000000,0.759555,0.790621,0.744894,0.709907,0.737439,0.744901,0.919548,-0.721494,0.266646,0.342079,0.064026,0.648328,0.324377,0.426556,0.671005
Shimmer,0.769063,0.703322,0.759581,0.797826,0.759555,1.000000,0.987258,0.987625,0.982835,0.950083,0.987626,0.722194,-0.835271,0.367430,0.447424,0.159954,0.654734,0.452025,0.507088,0.693771
Shimmer(dB),0.804289,0.716601,0.790652,0.839239,0.790621,0.987258,1.000000,0.963198,0.973751,0.960977,0.963202,0.744477,-0.827805,0.350697,0.410684,0.165157,0.652547,0.454314,0.512233,0.695058
APQ3,0.746625,0.697153,0.744912,0.763580,0.744894,0.987625,0.963198,1.000000,0.960070,0.896645,1.000000,0.716207,-0.827123,0.347617,0.435242,0.151124,0.610967,0.402243,0.467265,0.645377
APQ5,0.725561,0.648961,0.709927,0.786780,0.709907,0.982835,0.973751,0.960070,1.000000,0.949146,0.960072,0.658080,-0.813753,0.351148,0.399903,0.213873,0.646809,0.457195,0.502174,0.702456
APQ,0.758255,0.648793,0.737455,0.804139,0.737439,0.950083,0.960977,0.896645,0.949146,1.000000,0.896647,0.694019,-0.800407,0.364316,0.451379,0.157276,0.673158,0.502188,0.536869,0.721694


In [ ]:
#spread1, spread2 and PPE has a significant effect on the status. 

In [43]:
parkinsons_df.groupby('status').count()

,Jitter(%),Jitter(Abs),RAP,PPQ,DDP,Shimmer,Shimmer(dB),APQ3,APQ5,APQ,DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
status,,,,,,,,,,,,,,,,,,,
0.0,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
1.0,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147,147


In [ ]:
# 

In [44]:
X =  parkinsons_df.drop("status", axis=1)
y =  parkinsons_df.pop("status")

In [57]:
Dt_model=DecisionTreeClassifier(max_depth=4,criterion='entropy',random_state=0)

In [70]:
from sklearn.model_selection import GridSearchCV
parameter={'max_depth':np.arange(1,10), 'min_samples_leaf': np.arange(1,10), 'criterion' : ['entropy']}
GS=GridSearchCV(Dt_model,parameter,cv=3)
GS.fit(X,y)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), 'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9]), 'criterion': ['entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [54]:
GS.best_params_


{'criterion': 'entropy', 'max_depth': 3}

In [55]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [60]:
Dt_model1=DecisionTreeClassifier(max_depth=3,criterion='entropy',random_state=0)
Dt_model1.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [62]:
ypred=Dt_model1.predict(X_test)

In [63]:
accuracy =metrics.accuracy_score(y_test,ypred)
print(accuracy)

0.7966101694915254


In [66]:
confusionMatrix =metrics.confusion_matrix(ypred,y_test)
print(confusionMatrix)

[[10  9]
 [ 3 37]]


In [65]:
sum(y_test)

46.0

In [67]:
cr=metrics.classification_report(y_test,ypred)
print(cr)

              precision    recall  f1-score   support

         0.0       0.53      0.77      0.62        13
         1.0       0.93      0.80      0.86        46

   micro avg       0.80      0.80      0.80        59
   macro avg       0.73      0.79      0.74        59
weighted avg       0.84      0.80      0.81        59



In [71]:
def modelPerformanceAnalysis(X, y, model, test_size = 0.3, seed = 7):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
    model.fit(X_train, y_train)
    ### Unoptimized probability threshold based performance numbers
    # Train accuracy numbers
    trainPreds = model.predict(X_train)
    trainAccuracy = accuracy_score(y_train, trainPreds)
    trainRecall = recall_score(y_train, trainPreds)
    trainPrecision = precision_score(y_train, trainPreds)
    trainF1score = f1_score(y_train, trainPreds)
    trainResults=[X_train.shape[0], trainAccuracy, trainRecall, trainPrecision, trainF1score]
    # Test accuracy numbers
    testPreds = model.predict(X_test)
    testAccuracy = accuracy_score(y_test, testPreds)
    testRecall = recall_score(y_test, testPreds)
    testPrecision = precision_score(y_test, testPreds)
    testF1score = f1_score(y_test, testPreds)
    testResults=[X_test.shape[0], testAccuracy, testRecall, testPrecision, testF1score]
    
    #### Optimize probability threshold and calculate performance numbers
    y_predictProb = model.predict_proba(X_train)
    fpr, tpr, thresholds = roc_curve(y_train, y_predictProb[::,1])
    optimalF1 = 0
    optimalTh = 0
    for th in thresholds:
        preds = np.where(model.predict_proba(X_train)[:,1] > th, 1, 0)
        f1Score = f1_score(y_train, preds)
        if(optimalF1 < f1Score):
            optimalF1 = f1Score
            optimalTh = th

    resultsDf = pd.DataFrame( index=["dataSize", "accuracy", "recall", "precision", "f1_score"])
    # Optimal probability threshold based train accuracy numbers
    
    resultsDf['unOptimalTrainResults'] = trainResults
    resultsDf['unOptimalTestResults'] = testResults
    
    return resultsDf.round(2)

In [73]:
rf = RandomForestClassifier(n_estimators=100, max_depth=3, min_samples_leaf=2, random_state = 0)

In [76]:
from sklearn.model_selection import GridSearchCV
parameter={'n_estimators':np.arange(1,100)}
GS=GridSearchCV(rf,parameter,cv=3)
GS.fit(X,y)

KeyboardInterrupt: 

In [75]:
GS.best_params_


{'n_estimators': 31}

In [77]:
rf1 = RandomForestClassifier(n_estimators=31, max_depth=3, min_samples_leaf=2, random_state = 0)
modelPerformanceAnalysis(X, y , rf1)

/Users/apple/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/apple/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


,unOptimalTrainResults,unOptimalTestResults
dataSize,136.00,59.00
accuracy,0.93,0.83
recall,1.00,0.91
precision,0.92,0.88
f1_score,0.96,0.90


In [ ]:
# number of trees - 31 